# Computer Science Refershments Project 1

## Description

TODO

## Solution: 
_Divide and Conquer !_
Divide the problem when:
- change of segment in the theoretical trajectory
- intersection of the theoretical and experimental trajectories
- reach a _vertex_ in the experimental trajectory

How to calculate the error:
__The sum of the areas of the "Trapezoid" formed by two points on the theoretical trajectory and two points of the experimental trajectory__
o When two points from the experimental and theoretical trajectories are equals, the area is a triangle. 

In [1]:
#imports

import numpy as np

In [2]:
def trajectoryError(coordTh,coordExp):
    """
    arg1 coordTh : a list of tuples (x,y). The path which should be followed.
    arg2 coordExp: a list of tuples (x,y). The points received from the "indoors-gps"
    return: error, the total area difference betweeen the theoretical trajectory and the experimental one divided by the distance.
    """
    # compute total path length
    totalLength = path_length(coordTh)
    
    # create intermediary points the ortogonal-projection to the theoretical trajectory
    
    # create intermediary points at the interjections of the two trajectories
    
    # identify trapezoid -> two points on the theoretical trajectory, 2 points on the experiemental trajcetory
    
    # add the area of the trapezoid to the total error.
    
    # repeat until you the whole trajectory is computed
    
    error = area / distance
    
    return error


def seg_length(x_1,x_2):
    """
    arg1 x_1 : A tuple, the coordinates of the first point of the segment
    arg2 x_2 : A tuple, the coordinates of the second point of the segment
    return : A float, the length of the segment made up by x_1 and x_2
    """
    return(np.sqrt((x_1[0]-x_2[0])**2 + (x_1[1]-x_2[1])**2))


def path_length(path):
    """
    arg1 path : A table of tuples, each tuple representing the coordinates of a point of the path.
    return : leng. A float, 
    """
    assert len(path) > 1
    leng = 0
    for i in range(len(path)-1):
        leng += seg_length(path[i],path[i+1])
    return(leng)


def intersection(x_1, x_2, y_1, y_2):
    """
    arg1,2 x_i: A tuple, the coordinates of a point on the theoretical trajectory
    arg3,4 y_i: A tuple, the coordinates of a point on the experimental trajectory
    return: intersect. A tuple, the coordinates of the intersection or null.
    """
    intersect = None
    
    # express the line passing by x_1 and x_2
    
    return intersect


def ortogonalProjection(x_1, x_2, y):
    """
    arg1,2 x_i: A tuple, the coordinates of a point on the theoretical trajectory
    arg3 y: A tuple, the coordinates of a point on the experimental trajectory
    return: newPoint. A tuple, the coordinates of the orthogonal projection on y on the segment [x_1,x_2]
    """
    newPoint = (0,0)
    
    ## express the line passing by x_1 and x_2
    ## line = (x_2[1]-x_1[1])/(x_2[0]-x_1[0])
    
    return newPoint


def computeArea(x_1, x_2, y_1, y_2):
    """ 
    arg1,2 x_i: A tuple, the coordinates of a point on the theoretical trajectory
    arg3,4 y_i: A tuple, the coordinates of a point on the experimental trajectory
    return: area. An int, the area of the figure formed by the 4 coordinates
    """
    # Assert the arguments really form a trapezoid, with (x_1,y_1) being parallel with (x_2,y_2)
    ## Particular case where the xs are the same : vertical lines
    
    
    ## Other case : equality of the slopes
    
    
    # Computing the area
    area = 0
    
    # case 1: only two points -> return 0
    
    # case 2: if two points are equal -> triangle
    
    # case 3: all point distinct -> trapezoid
    
    return area